In [3]:
import * as torch from "./index.ts";


const a = new torch.Tensor([1, 2, 3], [3, 1]);
const b = new torch.Tensor([4, 5, 6], [3, 1]);
const c = torch.add(a, b);
console.log("a:", a.toString());
console.log("b:", b.toString());
console.log("c:", c.toString());

a: Tensor(shape=3,1, data=[1, 2, 3])
b: Tensor(shape=3,1, data=[4, 5, 6])
c: Tensor(shape=3,1, data=[5, 7, 9])
